In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import joblib
import keras
import cv2 as cv
sys.path.insert(0, "../packages/python")

2024-12-07 08:00:10.634352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733569210.653784    9514 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733569210.659062    9514 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 08:00:10.677257: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CSV_PATH = '../output/sam_uploaded_out/'
CROPS_PATH = '../output/cropped_cells_full/'
MODELS_PATH = '../models/'

In [3]:
def predict_cell(model_path, image_path, encoder_path=None):
    img = cv.imread(image_path)#, cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, (128, 128))
    _, model_extention = os.path.splitext(model_path)

    if model_extention != '.keras': 
        kmeans = joblib.load(model_path)

        if encoder_path != None: # Use clustering with encoder embeddings for detection
            img = np.expand_dims(img, axis=(0, -1))
            encoder = keras.models.load_model(encoder_path)
            feature = encoder.predict(img, verbose=0).astype(float)
            prediction = kmeans.predict(feature)[0]

        else: # Use clustering with color histogram for detection
            hist_predict = cv.calcHist(img, [0], None, [8], [0, 256]).flatten()
            hist_predict = np.array(hist_predict).reshape(1, -1)      
            hist_predict = hist_predict/255
            prediction = kmeans.predict(hist_predict)[0]

    else: #Use CNN for detection
        model = keras.models.load_model(model_path)
        img = np.array(img)
        img = img/255
        prediction = model.predict(np.expand_dims(img, 0),verbose = 0)[0][0]

    return True if prediction >= 0.5 else False

In [20]:
csvs = sorted(os.listdir(CSV_PATH))
crops = sorted(os.listdir(CROPS_PATH))
models = sorted(os.listdir(MODELS_PATH))

In [21]:
encoder = ''
for i, model in enumerate(models):
    if 'encoder' in model:
        encoder = models.pop(i)

In [ ]:
images=[]
for model in models:
    
    base_model, _ = os.path.splitext(model)
    output = f"../output/evaluated/{base_model}"
    if not os.path.exists(output): #Create dirs for each model used
        os.makedirs(output)

    for csv in csvs:
        base_name, _ = os.path.splitext(csv)
        images = sorted([crop for crop in crops if crop.startswith(base_name)])   
        df = pd.read_csv(os.path.join(CSV_PATH, csv))
        for idx, image in enumerate(images):
            print(f"Model: {model} - CSV: {csv} - Image: {idx + 1}/{len(images)} - {image}", end='\r')
            og_image_name, _ = os.path.splitext(image)
            cell_id = og_image_name.split('_')[2]
            is_cell = predict_cell(model_path=os.path.join(MODELS_PATH, model), image_path=os.path.join(CROPS_PATH, image))
            mask = (df['cell_id'] == int(cell_id)) & (df['image'] == base_name)
            df.loc[mask, 'is_cell'] = is_cell 
        df.to_csv(os.path.join(output, csv))

# print(images)